In [1]:
import pyrealsense2 as rs
print("pyrealsense2 is installed correctly!")


pyrealsense2 is installed correctly!


In [9]:
#Base code

In [67]:
import cv2
import numpy as np
import pyrealsense2 as rs
import os
import time

# Define the base directory to store images
base_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2"

# Get user input for scenario, place, object, and distance
scenario_num = input("Enter Scenario (1, 2, 3): ")
place_num = input("Enter Place (1, 2, 3): ")
object_num = input("Enter Object (1-10): ")
distance_val = input("Enter Distance (X, Y, Z): ")

# Create directory path dynamically based on input
output_dir = os.path.join(
    base_dir, f"Scenario_{scenario_num}", f"Place_{place_num}", f"Object_{object_num}", f"Distance_{distance_val}"
)

# Check if directory exists, create it if it doesn't
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Start RealSense pipeline
pipe = rs.pipeline()
cfg = rs.config()

# Enable RGB and depth streams
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 15)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 15)

# Start pipeline
pipe.start(cfg)

# Frame counter and sequence count
frame_count = 0
frames_to_capture = 33  # Minimum frames per sequence

print(f"Saving images to: {output_dir}")
print("Press [SPACE] to capture frame or [ESC] to exit...")

while True:
    time.sleep(.5)
    # Get frames
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    # Convert frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Apply colormap to depth image
    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET)

    # Show RGB and depth frames
    cv2.imshow('RGB', color_image)
    cv2.imshow('Depth', depth_cm)

    # Key press options
    key = cv2.waitKey(1) & 0xFF

    #if key == 32:  # Spacebar to capture frames
    if frame_count < frames_to_capture:
        # Save images to directory
        rgb_filename = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
        depth_filename = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")

        # Save RGB and depth images
        cv2.imwrite(rgb_filename, color_image)
        cv2.imwrite(depth_filename, depth_cm)

        frame_count += 1
        print(f"Captured frame {frame_count}/{frames_to_capture}...")
    else:
        break
        #print("Frame limit reached for this sequence.")
    
    '''elif key == 27:  # ESC to exit
        print("Exiting and stopping recording...")
        break'''

# Stop pipeline and release resources
pipe.stop()
cv2.destroyAllWindows()

print(f"Captured {frame_count}/{frames_to_capture} frames successfully. Images saved in '{output_dir}'.")


Enter Scenario (1, 2, 3):  3
Enter Place (1, 2, 3):  1
Enter Object (1-10):  10
Enter Distance (X, Y, Z):  16


Saving images to: F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2\Scenario_3\Place_1\Object_10\Distance_16
Press [SPACE] to capture frame or [ESC] to exit...
Captured frame 1/33...
Captured frame 2/33...
Captured frame 3/33...
Captured frame 4/33...
Captured frame 5/33...
Captured frame 6/33...
Captured frame 7/33...
Captured frame 8/33...
Captured frame 9/33...
Captured frame 10/33...
Captured frame 11/33...
Captured frame 12/33...
Captured frame 13/33...
Captured frame 14/33...
Captured frame 15/33...
Captured frame 16/33...
Captured frame 17/33...
Captured frame 18/33...
Captured frame 19/33...
Captured frame 20/33...
Captured frame 21/33...
Captured frame 22/33...
Captured frame 23/33...
Captured frame 24/33...
Captured frame 25/33...
Captured frame 26/33...
Captured frame 27/33...
Captured frame 28/33...
Captured frame 29/33...
Captured frame 30/33...
Captured frame 31/33...
Captured frame 32/33...
Captured frame 33/33...
Captured 33/33 frames successfully. I

In [ ]:
#calibrated test 1

In [3]:
import cv2
import numpy as np
import pyrealsense2 as rs
import os
import time

# Define the base directory to store images
base_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2"

# Get user input for scenario, place, object, and distance
scenario_num = input("Enter Scenario (1, 2, 3): ")
place_num = input("Enter Place (1, 2, 3): ")
object_num = input("Enter Object (1-10): ")
distance_val = input("Enter Distance (X, Y, Z): ")

# Create output directory
output_dir = os.path.join(
    base_dir, f"Scenario_{scenario_num}", f"Place_{place_num}", f"Object_{object_num}", f"Distance_{distance_val}"
)
os.makedirs(output_dir, exist_ok=True)

# Start RealSense pipeline
pipe = rs.pipeline()
cfg = rs.config()

# Enable RGB and depth streams
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 15)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 15)

# Start pipeline
profile = pipe.start(cfg)

# Align depth to color stream
align_to = rs.stream.color
align = rs.align(align_to)

# Optional: Create filters to smooth depth
spatial = rs.spatial_filter()
temporal = rs.temporal_filter()

# Capture loop settings
frame_count = 0
frames_to_capture = 33

print(f"Saving images to: {output_dir}")
print("Capturing frames...")

while frame_count < frames_to_capture:
    time.sleep(0.5)

    # Get aligned frames
    frames = pipe.wait_for_frames()
    aligned_frames = align.process(frames)

    aligned_depth_frame = aligned_frames.get_depth_frame()
    color_frame = aligned_frames.get_color_frame()

    if not aligned_depth_frame or not color_frame:
        print("Frame skip due to alignment issue.")
        continue

    # Apply filters to depth
    filtered_depth = spatial.process(aligned_depth_frame)
    filtered_depth = temporal.process(filtered_depth)

    # Convert frames to numpy arrays
    depth_image = np.asanyarray(filtered_depth.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Apply colormap for visualization
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET
    )

    # Ensure RGB and depth_colormap match in size
    if depth_colormap.shape != color_image.shape:
        depth_colormap = cv2.resize(depth_colormap, (color_image.shape[1], color_image.shape[0]))

    # Display both images
    cv2.imshow('RGB', color_image)
    cv2.imshow('Aligned Depth', depth_colormap)

    # Save images
    rgb_path = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
    depth_vis_path = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")
    depth_raw_path = os.path.join(output_dir, f"depth_raw_{frame_count:04d}.npy")

    cv2.imwrite(rgb_path, color_image)
    cv2.imwrite(depth_vis_path, depth_colormap)
    np.save(depth_raw_path, depth_image)

    print(f"Captured frame {frame_count+1}/{frames_to_capture}")
    frame_count += 1

    # ESC to break early
    if cv2.waitKey(1) & 0xFF == 27:
        print("Capture interrupted manually.")
        break

# Cleanup
pipe.stop()
cv2.destroyAllWindows()
print(f"Captured {frame_count} frames. Saved to '{output_dir}'.")


Enter Scenario (1, 2, 3):  1
Enter Place (1, 2, 3):  1
Enter Object (1-10):  10
Enter Distance (X, Y, Z):  11


Saving images to: F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2\Scenario_1\Place_1\Object_10\Distance_11
Capturing frames...
Captured frame 1/33
Captured frame 2/33
Captured frame 3/33
Captured frame 4/33
Captured frame 5/33
Captured frame 6/33
Captured frame 7/33
Captured frame 8/33
Captured frame 9/33
Captured frame 10/33
Captured frame 11/33
Captured frame 12/33
Captured frame 13/33
Captured frame 14/33
Captured frame 15/33
Captured frame 16/33
Captured frame 17/33
Captured frame 18/33
Captured frame 19/33
Captured frame 20/33
Captured frame 21/33
Captured frame 22/33
Captured frame 23/33
Captured frame 24/33
Captured frame 25/33
Captured frame 26/33
Captured frame 27/33
Captured frame 28/33
Captured frame 29/33
Captured frame 30/33
Captured frame 31/33
Captured frame 32/33
Captured frame 33/33
Captured 33 frames. Saved to 'F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2\Scenario_1\Place_1\Object_10\Distance_11'.


In [ ]:
#calibrated test 2

In [31]:
import cv2
import numpy as np
import pyrealsense2 as rs
import os
import time

# Setup
base_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2"
scenario_num = input("Enter Scenario (1, 2, 3): ")
place_num = input("Enter Place (1, 2, 3): ")
object_num = input("Enter Object (1-10): ")
distance_val = input("Enter Distance (X, Y, Z): ")

output_dir = os.path.join(
    base_dir, f"Scenario_{scenario_num}", f"Place_{place_num}", f"Object_{object_num}", f"Distance_{distance_val}"
)
os.makedirs(output_dir, exist_ok=True)

# Initialize RealSense
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 15)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 15)

profile = pipe.start(cfg)
align = rs.align(rs.stream.color)

# Filters (optional for smoother depth)
spatial = rs.spatial_filter()
temporal = rs.temporal_filter()

frame_count = 0
frames_to_capture = 33

print(f"Saving images to: {output_dir}")
print("Capturing frames...")

while frame_count < frames_to_capture:
    time.sleep(0.3)
    frames = pipe.wait_for_frames()
    aligned_frames = align.process(frames)

    color_frame = aligned_frames.get_color_frame()
    depth_frame = aligned_frames.get_depth_frame()

    if not color_frame or not depth_frame:
        print("Skipping frame due to capture error.")
        continue

    # Apply filters (optional)
    depth_frame = spatial.process(depth_frame)
    depth_frame = temporal.process(depth_frame)

    # Convert frames
    color_image = np.asanyarray(color_frame.get_data())
    depth_image = np.asanyarray(depth_frame.get_data())

    # Scale depth for display only (avoid resizing!)
    depth_display = cv2.convertScaleAbs(depth_image, alpha=0.03)  # More realistic contrast
    depth_colormap = cv2.applyColorMap(depth_display, cv2.COLORMAP_JET)

    # Crop depth_colormap to match RGB (only if needed)
    if depth_colormap.shape != color_image.shape:
        h, w = color_image.shape[:2]
        depth_colormap = depth_colormap[:h, :w]  # Crop instead of resize

    # Show
    cv2.imshow("RGB", color_image)
    cv2.imshow("Aligned Depth", depth_colormap)

    # Save outputs
    rgb_file = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
    depth_file = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")
            raw_file = os.path.join(output_dir, f"depth_raw_{frame_count:04d}.npy")

    cv2.imwrite(rgb_file, color_image)
    cv2.imwrite(depth_file, depth_colormap)
    np.save(raw_file, depth_image)

    print(f"Saved frame {frame_count + 1}/{frames_to_capture}")
    frame_count += 1

    if cv2.waitKey(1) & 0xFF == 27:
        print("Early exit via ESC.")
        break

pipe.stop()
cv2.destroyAllWindows()
print(f"✅ Captured {frame_count} frames in: {output_dir}")


Enter Scenario (1, 2, 3):  3
Enter Place (1, 2, 3):  3
Enter Object (1-10):  100
Enter Distance (X, Y, Z):  d11


Saving images to: F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2\Scenario_3\Place_3\Object_100\Distance_d11
Capturing frames...
Saved frame 1/33
Saved frame 2/33
Saved frame 3/33
Saved frame 4/33
Saved frame 5/33
Saved frame 6/33
Saved frame 7/33
Saved frame 8/33
Saved frame 9/33
Saved frame 10/33
Saved frame 11/33
Saved frame 12/33
Saved frame 13/33
Saved frame 14/33
Saved frame 15/33
Saved frame 16/33
Saved frame 17/33
Saved frame 18/33
Saved frame 19/33
Saved frame 20/33
Saved frame 21/33
Saved frame 22/33
Saved frame 23/33
Saved frame 24/33
Saved frame 25/33
Saved frame 26/33
Saved frame 27/33
Saved frame 28/33
Saved frame 29/33
Saved frame 30/33
Saved frame 31/33
Saved frame 32/33
Saved frame 33/33
✅ Captured 33 frames in: F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2\Scenario_3\Place_3\Object_100\Distance_d11


In [3]:
import cv2
import imageio
import numpy as np
import pyrealsense2 as rs
import os
import shutil

pipe = rs.pipeline()
cfg = rs.config()

cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

pipe.start(cfg)

while True:
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha = 0.5), cv2.COLORMAP_JET)
    
    cv2.imshow('rgb', color_image)
    cv2.imshow('depth', depth_cm)
    if cv2.waitKey(1) == ord('q'):
        break

pipe.stop()

In [ ]:
#Last updated code

In [6]:
import cv2
import imageio
import numpy as np
import pyrealsense2 as rs
import os
import shutil

pipe = rs.pipeline()
cfg = rs.config()

cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

pipe.start(cfg)

# Define the directory to store images
output_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2"

# Check if directory exists, create it if it doesn't
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

frame_count = 0

while True:
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha = 0.5), cv2.COLORMAP_JET)
    
    cv2.imshow('rgb', color_image)
    cv2.imshow('depth', depth_cm)





    # Increment frame counter
    frame_count += 1

    key = cv2.waitKey(1) & 0xFF
    if key == 32:
        # Save images to directory
        rgb_filename = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
        depth_filename = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")
        
        # Save RGB and depth images
        cv2.imwrite(rgb_filename, color_image)
        cv2.imwrite(depth_filename, depth_cm)      
    elif key == 27:
        break

pipe.stop()

RuntimeError: No device connected

In [ ]:
#for each folder 

In [11]:
#without fixed amount

In [ ]:
import cv2
import imageio
import numpy as np
import pyrealsense2 as rs
import os
from datetime import datetime

# Define the directory to store images
output_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images"

# Check if directory exists, create it if it doesn't
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Start RealSense pipeline
pipe = rs.pipeline()
cfg = rs.config()

# Enable RGB and depth streams
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start pipeline
pipe.start(cfg)

# Frame counter for unique filenames
frame_count = 0

while True:
    # Get frames
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    # Convert frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Apply colormap to depth image
    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET)

    # Show RGB and depth frames
    cv2.imshow('RGB', color_image)
    cv2.imshow('Depth', depth_cm)

    # Save images to directory
    rgb_filename = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
    depth_filename = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")

    # Save RGB and depth images
    cv2.imwrite(rgb_filename, color_image)
    cv2.imwrite(depth_filename, depth_cm)

    # Increment frame counter
    frame_count += 1

    # Break if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Stop pipeline and release resources
pipe.stop()
cv2.destroyAllWindows()

print(f"Images saved in '{output_dir}' directory.")


In [3]:
import pyrealsense2 as rs

ctx = rs.context()
devices = ctx.query_devices()
if len(devices) == 0:
    print("No device connected!")
else:
    for dev in devices:
        print("Device detected:", dev.get_info(rs.camera_info.name))


Device detected: Intel RealSense D455


In [ ]:
#For 100 frames

In [5]:
import cv2
import numpy as np
import pyrealsense2 as rs
import os

# Define the directory to store images
output_dir = r"F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images"

# Check if directory exists, create it if it doesn't
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Start RealSense pipeline
pipe = rs.pipeline()
cfg = rs.config()

# Enable RGB and depth streams
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Start pipeline
pipe.start(cfg)

# Frame counter for unique filenames
frame_count = 0
max_frames = 100  # Capture 100 frames and stop

while frame_count < max_frames:
    # Get frames
    frame = pipe.wait_for_frames()
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    # Check if frames are valid
    if not depth_frame or not color_frame:
        continue

    # Convert frames to numpy arrays
    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    # Apply colormap to depth image
    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET)

    # Show RGB and depth frames
    cv2.imshow('RGB', color_image)
    cv2.imshow('Depth', depth_cm)

    # Save images to directory
    rgb_filename = os.path.join(output_dir, f"rgb_frame_{frame_count:04d}.png")
    depth_filename = os.path.join(output_dir, f"depth_frame_{frame_count:04d}.png")

    # Save RGB and depth images
    cv2.imwrite(rgb_filename, color_image)
    cv2.imwrite(depth_filename, depth_cm)

    # Increment frame counter
    frame_count += 1

    # Break if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        print("Stopping recording...")
        break

# Stop pipeline and release resources
pipe.stop()
cv2.destroyAllWindows()

print(f"Captured {frame_count} frames. Images saved in '{output_dir}' directory.")


Captured 100 frames. Images saved in 'F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images' directory.


In [11]:
import cv2
import numpy as np

# Define the path to your depth image
depth_image_path = r'F:\1 KFUPM\3rd Semester\Computer Vision\Assignment 3\captured_images2\depth_frame_0119.png'

# Load the depth image correctly with IMREAD_UNCHANGED to preserve depth information
depth_image = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)

# Check if the image is loaded correctly
if depth_image is None:
    print("Error: Unable to load the depth image. Check the path.")
    exit()

# Print the shape to check dimensions
print(f"Depth image shape: {depth_image.shape}")

# Check if the image is still 3-channel
if len(depth_image.shape) == 3 and depth_image.shape[2] == 3:
    print("Error: Depth image is 3-channel (RGB), but it should be single-channel (grayscale).")
    # Convert to grayscale if needed (this may not be ideal for actual depth maps)
    depth_image = cv2.cvtColor(depth_image, cv2.COLOR_BGR2GRAY)
    print("Converted image to grayscale for processing.")

# Define camera parameters (update with your camera specs)
focal_length = 800  # Example focal length in pixels (replace with actual value)
baseline = 0.1  # Baseline in meters (distance between stereo cameras if applicable)

# Define the target pixel to calculate the distance (example: center of the image)
pixel_x, pixel_y = 320, 240  # Change based on the desired pixel

# Check if the pixel coordinates are within the image boundaries
height, width = depth_image.shape[:2]
if pixel_x >= width or pixel_y >= height:
    print(f"Pixel coordinates ({pixel_x}, {pixel_y}) are out of bounds.")
    exit()

# Get the depth value at the target pixel
depth_value = depth_image[pixel_y, pixel_x]

# Check if the depth value is valid and a scalar
if np.isscalar(depth_value) and (depth_value == 0 or np.isnan(depth_value)):
    print(f"No valid depth information at pixel ({pixel_x}, {pixel_y}).")
else:
    # If depth_value is an array with only one element, extract the value
    if isinstance(depth_value, np.ndarray) and depth_value.size == 1:
        depth_value = depth_value.item()

    # Ensure depth_value is a valid number
    if np.isscalar(depth_value):
        # Calculate the real-world distance using depth value
        real_world_distance = (depth_value * baseline) / focal_length

        # Print the calculated distance
        print(f"Distance at pixel ({pixel_x}, {pixel_y}) is {real_world_distance:.2f} meters.")
    else:
        print(f"Unexpected depth value format at pixel ({pixel_x}, {pixel_y}).")


Depth image shape: (480, 640, 3)
Error: Depth image is 3-channel (RGB), but it should be single-channel (grayscale).
Converted image to grayscale for processing.
Distance at pixel (320, 240) is 0.00 meters.
